In [31]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv("../df_approx.csv")
df.head()

,ts_name,fhat,PSD,freq,freq_approx_idx,freq_approx
0,H3,897560.000000,1.150877e+09,0.000000,0,0.000000
1,H3,32313.212902,1.491634e+06,1.428571,119,1.458190
2,H3,34207.839091,1.671680e+06,2.857143,149,2.865581
3,H3,25100.066233,9.000190e+05,4.285714,167,4.297856
4,H3,28363.946012,1.149305e+06,5.714286,180,5.759585


In [91]:
q_thresh = 0.9
cols = ['ts_1', 'ts_2', 'f_intersec']
df_f_overlap = pd.DataFrame(columns=cols)
for ts in tqdm(df.groupby("ts_name")):
    ts_name = ts[0]
    ts_df = ts[1]
    df_sub = df[df['ts_name']!=ts_name]
    
    thresh = np.quantile(ts_df['PSD'], q_thresh)
    core_df = ts_df[ts_df['PSD'] > thresh]
    fappr_idx_s = set(core_df['freq_approx_idx'])
    f_no = len(fappr_idx_s)
    
    ts_1 = []
    ts_2 = []
    f_intersec = []
    
    for ts_sub in df_sub.groupby("ts_name"):
        ts_name_sub = ts_sub[0]
        ts_df_sub = ts_sub[1]
        thresh_sub = np.quantile(ts_df_sub['PSD'], (f_no/ts_df_sub.shape[0]))
        core_df_sub = ts_df_sub[ts_df_sub['PSD']>thresh_sub]
        fappr_idx_sub_s = set(core_df_sub['freq_approx_idx'])
        no_intersection = len(fappr_idx_s & fappr_idx_sub_s)
        ts_1.append(ts_name)
        ts_2.append(ts_name_sub)
        f_intersec.append(no_intersection)
    data = {'ts_1': ts_1,
            'ts_2': ts_2,
            'f_intersec':f_intersec}
    df_f_sub = pd.DataFrame(data)
    df_f_sub.sort_values(by="f_intersec", ascending=False,
                        inplace=True)
    df_f_overlap = df_f_overlap.append(df_f_sub.iloc[:50,:]) # keep top 10 results
df_f_overlap.to_csv('df_f_overlap.csv', index=False)

  0%|          | 0/15 [00:00<?, ?it/s]

In [86]:
df_f_sub.sort_values('f_intersec', ascending=False, inplace=True)

In [92]:
df_f_overlap

,ts_1,ts_2,f_intersec
0,H1,H10,40
1,H1,H11,40
2,H1,H12,40
4,H1,H14,40
6,H1,H2,40
...,...,...,...
11,H9,H6,39
12,H9,H7,39
13,H9,H8,39
4,H9,H13,38
